## initial setup

In [ ]:
!nvidia-smi

Fri Apr  9 19:00:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !pip install -Uqq fastcore onnx onnxruntime sentencepiece seqeval rouge-score
    !pip install -Uqq --no-deps fastai ohmeow-blurr
    !pip install -Uqq transformers datasets wandb

     |████████████████████████████████| 61kB 5.1MB/s 
     |████████████████████████████████| 14.5MB 233kB/s 
     |████████████████████████████████| 4.1MB 51.9MB/s 
     |████████████████████████████████| 1.2MB 44.7MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 194kB 9.9MB/s 
     |████████████████████████████████| 61kB 8.2MB/s 
     |████████████████████████████████| 2.2MB 7.6MB/s 
     |████████████████████████████████| 194kB 17.8MB/s 
     |████████████████████████████████| 2.1MB 23.3MB/s 
     |████████████████████████████████| 3.3MB 56.4MB/s 
     |████████████████████████████████| 870kB 52.4MB/s 
     |████████████████████████████████| 112kB 58.0MB/s 
     |████████████████████████████████| 245kB 52.8MB/s 
     |████████████████████████████████| 133kB 53.9MB/s 
     |████████████████████████████████| 163kB 57.8MB/s 
     |████████████████████████████████| 102kB 12.8MB/s 
     |████████████████████████████████| 71kB 10.5MB/s 


In [ ]:
from fastai.text.all import *
from fastai.callback.wandb import *

In [ ]:
from transformers import *

from blurr.data.all import *
from blurr.modeling.all import *

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
task_name = 'squad'
model_name = 'distilbert-base-uncased'

max_len = 512
bs = 8
val_bs = 16

## Data

In [ ]:
# !wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
# !wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
squad_metric = load_metric(task_name)

In [ ]:
train_ds = load_dataset(task_name, split='train')
valid_ds = load_dataset(task_name, split='validation')

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/0fd9e01360d229a22adfe0ab7e2dd2adc6e2b3d6d3db03636a51235947d4c6e9. Subsequent calls will reuse this data.


Reusing dataset squad (/root/.cache/huggingface/datasets/squad/plain_text/1.0.0/0fd9e01360d229a22adfe0ab7e2dd2adc6e2b3d6d3db03636a51235947d4c6e9)


In [ ]:
train_ds[0]

{'answers': {'answer_start': [515], 'text': ['Saint Bernadette Soubirous']},
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'id': '5733be284776f41900661182',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'title': 'University_of_Notre_Dame'}

In [ ]:
# train_ds = train_ds.select(range(1000))
valid_ds = valid_ds.select(range(200))
len(train_ds), len(valid_ds)

(87599, 200)

In [ ]:
hf_arch, hf_config, hf_tokenizer, hf_model = BLURR_MODEL_HELPER.get_hf_objects(model_name, tokenizer_cls=AutoTokenizer, model_cls=AutoModelForQuestionAnswering, tokenizer_kwargs={'max_len':max_len}, )

based on huggingface example https://github.com/huggingface/transformers/blob/master/examples/question-answering/run_qa.py

TODO: check on squad_v2

In [ ]:
# Preprocessing the datasets.
# Preprocessing is slighlty different for training and evaluation.
column_names = train_ds.column_names

question_column_name = "question" #if "question" in column_names else column_names[0]
context_column_name = "context" #if "context" in column_names else column_names[1]
answer_column_name = "answers" #if "answers" in column_names else column_names[2]

# Padding side determines if we do (question|context) or (context|question).
tokenizer = hf_tokenizer
pad_on_right = tokenizer.padding_side == "right"

max_seq_length = min(max_len, tokenizer.model_max_length)

# Training preprocessing
def prepare_train_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples[question_column_name if pad_on_right else context_column_name],
        examples[context_column_name if pad_on_right else question_column_name],
        truncation="only_second" if pad_on_right else "only_first",
        return_attention_mask=False,
        max_length=max_seq_length,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding=False,
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples[answer_column_name][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


In [ ]:
train_ds = train_ds.map(
    prepare_train_features,
    batched=True,
    num_proc=2,
    remove_columns=column_names
)

In [ ]:
len(train_ds)

87714

In [ ]:
train_ds[0].keys()

dict_keys(['end_positions', 'input_ids', 'start_positions'])

In [ ]:
x, start_, end_ = train_ds[0]['input_ids'], train_ds[0]['start_positions'], train_ds[0]['end_positions']

In [ ]:
hf_tokenizer.decode(x[start_:end_], skip_special_tokens=True)

'saint bernadette soubiro'

In [ ]:
class SQUADBeforeBatchTransform(Transform):
    def __init__(self, hf_arch, hf_config, hf_tokenizer, hf_model, pad_to_multiple_of=None,
                 max_length=None, padding=True, truncation=True, is_split_into_words=False,
                 tok_kwargs={}, **kwargs):
        store_attr()

    def encodes(self, samples):
        samples = L(samples)
        # grab inputs
        if (is_listy(samples[0][0]) and not self.is_split_into_words):
            batch = list(zip(samples.itemgot(0, 0), samples.itemgot(0, 1)))
        else:
            batch = samples.itemgot(0).items

        batch = self.hf_tokenizer.pad(batch,
                                     return_tensors='pt',
                                     pad_to_multiple_of = self.pad_to_multiple_of)
        del batch['start_positions'], batch['end_positions']
        # update samples with tokenized inputs (e.g. input_ids, attention_mask, etc...)
        d_keys = batch.keys()
        updated_samples= [ (*[{k: batch[k][idx] for k in d_keys}], *sample[1:])
                          for idx, sample in enumerate(samples) ]

        return updated_samples

In [ ]:
before_batch_tfm = SQUADBeforeBatchTransform(hf_arch, hf_config, hf_tokenizer, hf_model,
                                             max_length=max_len, 
                                             #tok_kwargs={ 'return_special_tokens_mask': True }
                                             )

In [ ]:
train_ds.column_names

['end_positions', 'input_ids', 'start_positions']

In [ ]:
blocks = (
    HF_TextBlock(before_batch_tfm=before_batch_tfm, input_return_type=HF_QuestionAnswerInput), 
    CategoryBlock(vocab=list(range(max_len))),
    CategoryBlock(vocab=list(range(max_len)))
)

def get_x(x):
    return (x['question'], x['context']) if (hf_tokenizer.padding_side == 'right') else (x['context'], x['question'])

dblock = DataBlock(blocks=blocks,
                   get_y=[ItemGetter('start_positions'), ItemGetter('end_positions')],
                   splitter=RandomSplitter(), # TODO use valid ds properly
                   n_inp=1)

In [ ]:
# dblock.summary(train_ds)

In [ ]:
%%time
dls = dblock.dataloaders(train_ds, bs=8, val_bs=16)

CPU times: user 1min 18s, sys: 2.13 s, total: 1min 20s
Wall time: 1min 27s


In [ ]:
dls.one_batch()

({'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          ...,
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0'),
  'input_ids': tensor([[ 101, 2043, 2001,  ...,    0,    0,    0],
          [ 101, 4998, 2013,  ...,    0,    0,    0],
          [ 101, 2006, 2054,  ...,    0,    0,    0],
          ...,
          [ 101, 2054, 2003,  ...,    0,    0,    0],
          [ 101, 2129, 4206,  ..., 1007, 1012,  102],
          [ 101, 2073, 2515,  ...,    0,    0,    0]], device='cuda:0')},
 TensorCategory([107,  65, 120,  81,  80,  52,  21,  90], device='cuda:0'),
 TensorCategory([107,  71, 121,  81,  84,  58,  24,  98], device='cuda:0'))

In [ ]:
# Validation preprocessing
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples[question_column_name if pad_on_right else context_column_name],
        examples[context_column_name if pad_on_right else question_column_name],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_seq_length,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding=False,
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
    # corresponding example_id and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
valid_ds = valid_ds.map(prepare_validation_features,
                        batched=True,
                        num_proc=2,
                        remove_columns=column_names)

In [ ]:
# valid_ds[0]

In [ ]:
valid_ds.column_names

['attention_mask', 'example_id', 'input_ids', 'offset_mapping']

## Training

Won't be needed when fastai `one_batch` is fixed

In [ ]:
def _to_device(e, device):
    if hasattr(e, 'to'): return e.to(device)
    elif isinstance(e, dict):
        for _, v in e.items():
            if hasattr(v, 'to'): v.to(device)
        return {k:(v.to(device) if hasattr(v, 'to') else v) for k, v in e.items()}

In [ ]:
@patch
def one_batch(self:Learner, i, b):
        self.iter = i
        b_on_device = tuple(_to_device(e, self.dls.device) for e in b) if self.dls.device is not None else b
        self._split(b_on_device)
        self._with_events(self._do_one_batch, 'batch', CancelBatchException)

In [ ]:
def process_example(start_indexes, end_indexes, start_probs, end_probs, max_answer_length):
    # process single example
    candidates = []
    for start_index in start_indexes:
        for end_index in end_indexes:
            if (start_index > end_index or
                end_index - start_index + 1 > max_answer_length or
                start_index >= len(start_probs) or
                end_index >= len(start_probs)):
                continue
            # collect plausable answers in candidates
            candidates.append({'prob':start_probs[start_index] * end_probs[end_index],
                            'start_index':start_index,
                            'end_index':end_index})
    # order candidates by probability
    candidates = sorted(candidates, key= lambda x : x['prob'], reverse=True)

    #TODO add option for null answerfor squad_v2
    # simple case answer always exists
    if candidates:
        best = candidates[0]
        best_answer = (best['start_index'], best['end_index'])
    else:
        best_answer = (0, 0)
    return best_answer

In [ ]:
def qa_exact_match(preds, *targs, n_best_size=10, max_answer_length=20):
    start_targs, end_targs = targs
    start_probs, end_probs = preds[0].softmax(-1), preds[1].softmax(-1)
    start_indexes = start_probs.argsort(descending=True)[:, :n_best_size].tolist()
    end_indexes = end_probs.argsort(descending=True)[:, :n_best_size].tolist()
    bs = start_targs.size(0)
    # iterate over samples
    best_preds = [process_example(start_indexes[i], end_indexes[i], start_probs[i], end_probs[i], max_answer_length) for i in range(bs)]
    best_preds = tensor(best_preds).to(start_targs.device)
    return ((best_preds[:,0] == start_targs)*(best_preds[:,1] == end_targs)).float().mean()

In [ ]:
exact_match = AvgMetric(partial(qa_exact_match, n_best_size=10, max_answer_length=20))

In [ ]:
model = HF_BaseModelWrapper(hf_model)
learn = Learner(dls,
                model,
                opt_func=RAdam,
                loss_func=MultiTargetLoss(),
                cbs=[HF_QstAndAnsModelCallback],
                metrics=exact_match,
                splitter=hf_splitter).to_fp16()

# learn.blurr_summary()

In [ ]:
# learn.show_training_loop()

### simple finetuning

In [ ]:
learn.fit_one_cycle(5, 2e-4,)# cbs=WandbCallback(log_preds=False, log_model=False))

epoch,train_loss,valid_loss,qa_exact_match,time
0,3.023777,3.399067,0.454395,18:58
1,3.282606,3.313993,0.440600,18:49


epoch,train_loss,valid_loss,qa_exact_match,time
0,3.023777,3.399067,0.454395,18:58
1,3.282606,3.313993,0.440600,18:49
2,2.435078,2.988558,0.490651,18:59
3,1.524035,2.820082,0.526850,19:08
4,0.677437,3.354660,0.525653,19:17


## Fin